## Bibliothèque permanente

Comme nous avons vu dans le chapitre précédent, SAS a besoin d'importer les données de n'importe quel format afin que l'utilisateur puisse appliquer les analyses souhaitées. Toutesfois, afin d'être plus productif, il est plus simple de conserver ces données quelque part où nous ne sommes pas obligés de refaire cette opération d'importation d'un jeu de données à chaque fois que nous en avons besoin. Surtout lorsque nous y effectuons des changements tels que des ajouts de variables ou des calculs qui prennent du temps à exécuter.

SAS nous permet de sauvegarder ces données dans ce qu'on appelle une **bibliothèque** ou (_**library**_)

Mais d'abord, rappelons-nous comment les données ont organisés dans SAS. Ci-dessous, un parallèle avec ce que nous sommes habitués de voir avec des tableurs tel qu’Excel de Microsoft ou Calc de [LibreOffice](https://www.libreoffice.org/discover/calc/);



<style type="text/css">
	table.tableizer-table {
		font-size: 12px;
		border: 1px solid #CCC; 
		font-family: Arial, Helvetica, sans-serif;
	} 
	.tableizer-table td {
		padding: 4px;
		margin: 3px;
		border: 1px solid #CCC;
	}
	.tableizer-table th {
		background-color: #104E8B; 
		color: #FFF;
		font-weight: bold;
	}
</style>
<table class="tableizer-table">
<thead><tr class="tableizer-firstrow"><th>Excel (ou autre)</th><th>SAS</th></tr></thead><tbody>
 <tr><td>feuille</td><td>data set</td></tr>
 <tr><td>colonne</td><td>variable</td></tr>
 <tr><td>ligne</td><td>observation</td></tr>
</tbody></table>

Lorsqu'on créait un jeu de données avec la procédure `data` comme suit;

In [ ]:
filename mesDos 'data/emprunt_bancaire.csv';
data loan;
    infile mesDos dsd
     firstobs=2;
    input Loan_ID $ loan_status $ Principal terms age education $ Gender $;
run;

En réalité, SAS sauvegarde les données sous le format `work.loan`, le `work` qui précède `loan` est le nom de la bibliothèque (_library reference_) temporaire que SAS crée par défaut. SAS y insère tous les jeux de données si l'utilisateur n'en définit pas une.

D'ailleurs, dans tous les exemples que nous avons réalisés jusqu'à présent, nous avons créé des jeux de données dans une bibliothèque temporaire nommée `WORK`. Vous pouvez le vérifier en affichant les données comme suit:


In [9]:
proc print data=work.loan noobs;
run;

Remarquez qu'ici nous ajoutons `work.` même si nous ne le précisons pas à la création du jeu de données.

## Comment créer une bibliothèque permanente?

Nous créons une bibliothèque en donnant simplement le nom de cette dernière dans la commande `LIBNAME`  et de spécifier dans quel répertoire nous voulons la stocker.

La longueur du nom de la bibliothèque ne doit pas dépasser 8 caractères et doit respecter les règles de validation de SAS (caractères spéciaux ...etc.)

Allons-y avec un exemple où je veux créer une bibliothèque permanente appelée `act3035` dans le répertoire `data`. Dans cet exemple, j'aurais pu mettre ma bibliothèque permanente dans un autre répertoire qui serait `repertoire/sous_repertoire/data`

In [10]:
libname act3035 'data'; /* repertoire/sous_repertoire/data */
data act3035.notes_examens;
infile datalines dsd;
    length code_perm $ 50;
    input code_perm $ examen1-examen3;
    datalines;
    "Diane Rossi",67,68,70
    "Stéphanie de la Gonzalez",72,81,87
    "Thierry Lacroix",73,87,86
    "Jérôme Blanc-Moreno",73,66,77
    "Danielle Le Guyon",81,73,84
    "Alfred Adam",73,79,82
    "Chantal Mahe",75,77,69
    "Eugène Dupont",81,80,83
    "Bertrand Pages Le Didier",86,77,75
    "Guy Gillet de la Valette",78,69,80
    ;
proc print data=act3035.notes_examens nobs;
run;

Comme expliqué en classe, un des avantages de prendre l'habitude de travailler avec des bibliothèques permanentes, est de ne pas refaire le travaille de [ETL](https://www.sas.com/en_us/insights/data-management/what-is-etl.html) (extraire les données, les transformer et les importer).

En plus, si nous avons un (ou plusieurs) **petit** ensemble de données, ce n'est pas très pénalisant de travailler avec les bibliothèques temporaires, et nous pouvons aisément nous en sortir avec la taille de la mémoire de nos ordinateurs. Toutefois, dès que nous avons des bases de données plus importantes, comme c'est le cas en pratique, il est très pertinent de créer des bibliothèques permanentes et d'y stocker nos ensembles de données. 

## Revoir la librairie

libname lib_prof "/home/ec2913580/my_courses/nmeraihi1/data";

/* par exemple vus pouvez afficher ces données */
proc print data=lib_prof.eq_2 (firstobs = 2 obs = 5) noobs;
run;

/* on peut  même prendre une copie de cette librarie lib_prof et
faire une copie dans work*/
 data copie_eq2;
 	set lib_prof.eq_2;
 	where type_prof like "%eur" and sexe like "M";
 	drop numeropol cheveux;
 	if cout1>10000;
 run;
 
/*  toutefois, on ne peut rien mettre de-dans*/
 data lib_prof.copie_eq2;
 	set lib_prof.eq_2;
 	where type_prof like "%eur" and sexe like "M";
 	drop numeropol cheveux;
 	if cout1>10000;
 run;
 
 
/*  On peut touefois créer notre copie de données dans une librarire personnelle 
même si elle pointe vers VOTRE propre répertoire qui est vide au départ*/






 
/* Si vous voulez faire des changement, vous pouvez copier toute les
données dans un répertoire personnel qui vous appartient en mode lecture
ET écriture comme on l'a vue en classe*/



## Description de la bibliothèque

Nous pouvons voir la description des bibliothèques permanentes que nous avons créées avec la commande `PROC CONTENTS`.

In [7]:
proc contents data=act3035.notes_examens;
run;

Nous pouvons aussi faire une description de tous les ensembles de données avec l'option `_ALL_` contenus dans une bibliothèque

In [8]:
title "Listing All the SAS Data Sets in a Library";
proc contents data=act3035._all_ nods;
run;

## Données manquantes

Lorsqu'une valeur de donnée est inconnue, alors nous avons une donnée manquante. 
Une donnée manquante est une donnée valide, elle est affichée par un espace (vide) dans une variable (colonne) de type caractère, et par un point lorsqu'il s'agit d'une variable (colonne) numérique.

## Nommer des variables dans SAS

SAS possède une convention afin de donner des noms de variables. Nous pouvons nommer nos variables avec une combinaison de 1 à 32 caractères, numériques, _underscore_ ou une combinaison de ces trois derniers. Les lettres peuvent être en minuscule ou majuscule.

### Références
Guide, A. Programmer’S. "Learning SAS® by Example."